In [94]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [95]:
# import excel file from local drive 
file_path = r"C:\Users\phuya\OneDrive\Pet_Project\Country_Portfolio\Capital Flow Datasets (Koepke Paetzold 2020) - Version as of January 2023.xlsx"


In [96]:
# creating a nice dataframe from semi-structured multi header Data. 
df = pd.read_excel(file_path, sheet_name = "Monthly Capital Flow Dataset", header = 2).set_index("time").T
df = df.iloc[:,:-3] # excluding the redundent columns. 
df.columns = ["country_code", "Indicies"] + list(df.columns[2:]) # renaming the columns 
df.reset_index(drop=False, inplace=True)
df.rename(columns={"index":"Countries"}, inplace=True)
df["Countries"] = df["Countries"].apply(lambda x: np.nan if "Unnamed" in x else x)
df["Countries"] = df["Countries"].ffill()


In [98]:
# Using Melt function to transform the Data in more accessable way

df =  df.melt(id_vars= ["Countries", "country_code", "Indicies"], # variables which  retain ther position as it is.
              var_name="Date",
              value_name= "Value") # nameing the rest of the column which is getting un-pivoted 

In [99]:
# Date Transformation
df["Date"] = pd.to_datetime(df["Date"], format= "%Y%b")
df["Year"] = pd.DatetimeIndex(df["Date"]).year # is useful for creating interactive app. 
df["Month"] = pd.DatetimeIndex(df["Date"]).month # is useful for creating interactive app. 


In [100]:
# retaining just the name of country, to merge with alpha code dataset
df["Countries"] = df["Countries"].replace(to_replace=r"\(.*\)", value="", regex=True)
df["Countries"] = df["Countries"].str.strip()

In [101]:
# importing the dataframe with apha codes and mering with working dataframe. 
df_Alppha_Codes = pd.read_csv("alpha_country_codes.csv")
df_Alppha_Codes.rename(columns={"Country": "Countries"}, inplace=True)
df= pd.merge(df, df_Alppha_Codes[["Countries", "Alpha-3 code"]], how="left", on="Countries")


In [103]:
# Manually filing the missing countries while merging two dataframes. 
missing_countries = df[df["Alpha-3 code"].isna()].Countries.unique()
df_Alppha_Codes[df_Alppha_Codes["Countries"].str.contains("Macedonia")]

array(['Emerging Markets', 'Korea', 'Philippines', 'Czech Republic',
       'Slovak Republic', 'North Macedonia'], dtype=object)

In [105]:
df.loc[df["Countries"] == "Korea", "Alpha-3 code"] = df.loc[df["Countries"] == "Korea", "Alpha-3 code"].fillna("KOR")
df.loc[df["Countries"] == "Philippines", "Alpha-3 code"] = df.loc[df["Countries"] == "Philippines", "Alpha-3 code"].fillna("PHL")
df.loc[df["Countries"] == "Czech Republic", "Alpha-3 code"] = df.loc[df["Countries"] == "Czech Republic", "Alpha-3 code"].fillna("CZE")
df.loc[df["Countries"] == "Slovak Republic", "Alpha-3 code"] = df.loc[df["Countries"] == "Slovak Republic", "Alpha-3 code"].fillna("SVK")
df.loc[df["Countries"] == "North Macedonia", "Alpha-3 code"] = df.loc[df["Countries"] == "North Macedonia", "Alpha-3 code"].fillna("MKD")



In [108]:
#pd.read_csv("Clean_Capital_Flow_data.csv")